In [14]:
%matplotlib inline
import numpy as np
from sklearn import svm
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [5]:
with open("/home/christopher/data/ml/digit-recognizer/train.csv", "r") as data:
    fullTrain = np.loadtxt(data, delimiter=",", skiprows=1, dtype="float64")
with open("/home/christopher/data/ml/digit-recognizer/test.csv", "r") as data:
    fullTest = np.loadtxt(data, delimiter=",", skiprows=1, dtype="float64")

In [48]:
train = fullTrain
test = fullTest

In [49]:
trainResults = train[:,0]
trainData = train[:,1:]
testData = test

# Scaling

We need to scale our data to have mean 0, variance 1. 

In [50]:
scaler = preprocessing.StandardScaler().fit(trainData)
scaledTrainData = scaler.transform(trainData)
scaledTestData = scaler.transform(testData)


# Fit

Fit a support vector classification model to our data then predict our test data from that.

In [ ]:
clf = svm.SVC()#gamma=0.001, kernel="sigmoid")   
_ = clf.fit(scaledTrainData, trainResults)

In [46]:
res = clf.predict(testData)

In [27]:
with open("results.csv", "w") as f:
    f.write("ImageId,Label\n")
    for i, r in enumerate(res):
        f.write("{},{}\n".format(i+1, int(r)))